In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Define the Residual Block
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x

    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # If the shape of the input tensor is different from the shape of the output tensor,
    # add a convolutional layer to the shortcut path
    if shortcut.shape[-1] != filters or stride != 1:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)

    return x

# Define the ResNet model
def build_resnet(input_shape=(28, 28, 1), num_classes=10):
    inputs = layers.Input(shape=input_shape)

    # Initial convolutional layer
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, 64, stride=2)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)

    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully connected layer
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=x, name='resnet')
    return model




# Build and compile the model
input_shape = (28, 28, 1)
num_classes = 10
model_m = build_resnet(input_shape, num_classes)
model_m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Train the model
model_m.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
938/938 [==============================] - 47s 25ms/step - loss: 0.1871 - accuracy: 0.9463 - val_loss: 0.1062 - val_accuracy: 0.9676
Epoch 2/10
938/938 [==============================] - 22s 24ms/step - loss: 0.0734 - accuracy: 0.9787 - val_loss: 0.1607 - val_accuracy: 0.9609
Epoch 3/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0563 - accuracy: 0.9839 - val_loss: 0.5319 - val_accuracy: 0.9212
Epoch 4/10
938/938 [==============================] - 23s 24ms/step - loss: 0.0519 - accuracy: 0.9848 - val_loss: 0.0423 - val_accuracy: 0.9864
Epoch 5/10
938/938 [==============================] - 23s 24ms/step - loss: 0.0544 - accuracy: 0.9841 - val_loss: 0.1359 - val_accuracy: 0.9600
Epoch 6/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0409 - accuracy: 0.9879 - val_loss: 0.0333 - val_accuracy: 0.9895
Epoch 7/10
938/938 [==============================] - 23s 24ms/step - l

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
# Evaluate the model on MNIST testing samples
y_pred_m = model_m.predict(x_test)
y_pred_classes_m= np.argmax(y_pred_m, axis=1)
y_true_m = np.argmax(y_test, axis=1)


# Compute confusion matrix, precision, recall, and F1 score
confusion_m= confusion_matrix(y_true_m, y_pred_classes_m)
precision_m = precision_score(y_true_m, y_pred_classes_m, average='weighted')
recall_m = recall_score(y_true_m, y_pred_classes_m, average='weighted')
f1_m = f1_score(y_true_m, y_pred_classes_m, average='weighted')

print("(MNIST):")
print("Confusion Matrix :")
print(confusion_m)
print(f"Precision : {precision_m}")
print(f"Recall : {recall_m}")
print(f"F1 Score : {f1_m}")

313/313 [==============================] - 2s 5ms/step
(MNIST):
Confusion Matrix :
[[ 972    0    1    0    0    0    6    1    0    0]
 [   1 1101    7    1    4    0    4    2   15    0]
 [   1    0 1025    0    0    0    6    0    0    0]
 [   0    0   15  979    0    0    2    2   12    0]
 [   0    0    0    0  971    0   10    0    0    1]
 [   3    0    0    3    0  849   11    0   26    0]
 [   3    1    0    0    0    1  952    0    1    0]
 [   0    1   14    0    5    0    0 1004    0    4]
 [   1    0    6    0    0    0    2    0  964    1]
 [   0    0    2    0   10    2    0    1   21  973]]
Precision : 0.9797298439696739
Recall : 0.979
F1 Score : 0.9790857763924048


In [ ]:
model_m.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_40 (Conv2D)          (None, 14, 14, 64)           3200      ['input_9[0][0]']             
                                                                                                  
 batch_normalization_40 (Ba  (None, 14, 14, 64)           256       ['conv2d_40[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_24 (Activation)  (None, 14, 14, 64)           0         ['batch_normalization_40[

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

# Load and preprocess the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build and compile the model

model_f = build_resnet(input_shape, num_classes)
model_f.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_f.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/10
938/938 [==============================] - 33s 24ms/step - loss: 0.4743 - accuracy: 0.8335 - val_loss: 0.4143 - val_accuracy: 0.8515
Epoch 2/10
938/938 [==============================] - 21s 23ms/step - loss: 0.3336 - accuracy: 0.8789 - val_loss: 0.3416 - val_accuracy: 0.8836
Epoch 3/10
938/938 [==============================] - 22s 24ms/step - loss: 0.3010 - accuracy: 0.8910 - val_loss: 0.3333 - val_accuracy: 0.8870
Epoch 4/10
938/938 [==============================] - 22s 23ms/step - loss: 0.2654 - accuracy: 0.9017 - val_loss: 0.3246 - val_accuracy: 0.8816
Epoch 5/10
938/938 [==============================] - 21s 23ms/step - loss: 0.2667 - accuracy: 0.9038 - val_loss: 0.6511 - val_accuracy: 0.7719
Epoch 6/10
938/938 [==============================] - 22s 23ms/step - loss: 0.2521 - accuracy: 0.9074 - val_loss: 0.3483 - val_accuracy: 0.8681
Epoch 7/10
938/938 [==============================] - 21s 22ms/step - los

In [ ]:
# Evaluate the model on MNIST testing samples
y_pred_f = model_f.predict(x_test)
y_pred_classes_f= np.argmax(y_pred_f, axis=1)
y_true_f = np.argmax(y_test, axis=1)


# Compute confusion matrix, precision, recall, and F1 score
confusion_f= confusion_matrix(y_true_f, y_pred_classes_f)
precision_f = precision_score(y_true_f, y_pred_classes_f, average='weighted')
recall_f = recall_score(y_true_f, y_pred_classes_f, average='weighted')
f1_f = f1_score(y_true_f, y_pred_classes_f, average='weighted')

print("(Fashion MNIST):")
print("Confusion Matrix :")
print(confusion_f)
print(f"Precision : {precision_f}")
print(f"Recall : {recall_f}")
print(f"F1 Score : {f1_f}")

313/313 [==============================] - 2s 5ms/step
(Fashion MNIST):
Confusion Matrix :
[[854   1  12  10   2   0 117   0   4   0]
 [  2 989   0   3   2   0   3   0   1   0]
 [ 28   2 805   5  87   0  73   0   0   0]
 [ 29   5   4 906  19   0  37   0   0   0]
 [  1   0  51  36 863   0  48   0   1   0]
 [  0   0   0   0   0 982   0  13   0   5]
 [111   4  55  18  68   0 737   0   7   0]
 [  0   0   0   0   0   9   0 984   0   7]
 [  9   2   2   7   1   2   5   4 968   0]
 [  0   0   1   0   0   5   0  45   0 949]]
Precision : 0.904706871614678
Recall : 0.9037
F1 Score : 0.9039256916225448
